In [1]:
import os
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import dataset
import schedulers

import tensorflow as tf
assert tf.config.list_physical_devices('GPU')

In [3]:
BATCH_SIZE = 32

LETTERS_SIZE = len(dataset.letters_table)
NIQQUD_SIZE = len(dataset.niqqud_table)
DAGESH_SIZE = len(dataset.dagesh_table)
SIN_SIZE = len(dataset.sin_table)

def build_model(UNITS=380):  # EMBED_DIM=28
    inp = keras.Input(batch_shape=(None, None), batch_size=BATCH_SIZE)
    layer = layers.Embedding(LETTERS_SIZE, UNITS, mask_zero=True)(inp)
    
    bidi = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True), merge_mode='sum')
    layer = bidi(layer)
    layer = layers.add([layer, bidi(layer)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(layer)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(layer)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(layer)),
    ]
    model = keras.Model(inputs=inp, outputs=outputs)

    return model

model = build_model()

model.summary()
model.save_weights('./checkpoints/uninit')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 380)    16720       input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, None, 380)    2313440     embedding_1[0][0]                
                                                                 bidirectional_1[0][0]            
__________________________________________________________________________________________________
add_1 (Add)                     (None, None, 380)    0           bidirectional_1[0][0]      

In [7]:
def accuracy(y_true, y_pred):
    K = keras.backend
    f = K.floatx()
    # convert dense predictions to labels
    y_pred_labels =  K.cast(K.argmax(y_pred, axis=-1), f)
    
    res = K.cast(K.equal(y_true, y_pred_labels), f)
    return K.sum(res) / K.sum(K.cast(K.not_equal(y_true, 0), f))


def fit(train_validation, scheduler=None, verbose=1, lr=1e-4):
    train, valid = train_validation
    model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=lr), metrics=[accuracy])
    callbacks = []
    if isinstance(scheduler, schedulers.CircularLearningRate):
        scheduler.set_dataset(train, BATCH_SIZE)
    if scheduler:
        callbacks.append(scheduler)
        
    x  = train.normalized
    vx = valid.normalized
    
    y  = {'N': train.niqqud, 'D': train.dagesh, 'S': train.sin }
    vy = {'N': valid.niqqud, 'D': valid.dagesh, 'S': valid.sin }
    
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=1, verbose=verbose, callbacks=callbacks)


MAXLEN = 64
def load_data(source, maxlen=MAXLEN, validation=0.1):
    filenames = [os.path.join('texts', f) for f in source]
    train, valid = dataset.load_data(filenames, validation, maxlen=maxlen)
    return train, valid

In [8]:
data_mix = load_data(['poetry', 'rabanit', 'pre_modern'])

In [31]:
model.load_weights('./checkpoints/uninit')
history = fit(data_mix, scheduler=schedulers.CircularLearningRate(30e-4, 80e-4, 1e-4))
model.save_weights('./checkpoints/mix')

4415/4415 [==============================] - 273s 62ms/step - loss: 0.3155 - N_loss: 0.2434 - D_loss: 0.0634 - S_loss: 0.0087 - N_accuracy: 0.9148 - D_accuracy: 0.9754 - S_accuracy: 0.9977 - val_loss: 0.3062 - val_N_loss: 0.2383 - val_D_loss: 0.0610 - val_S_loss: 0.0069 - val_N_accuracy: 0.9219 - val_D_accuracy: 0.9763 - val_S_accuracy: 0.9982


In [17]:
data_modern = load_data(validation=0.1, source=['modern'])

In [33]:
vals = []
for i in range(15):
    model.load_weights('./checkpoints/mix')
    data_modern = load_data(validation=0.1, source=['modern'])
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(50e-4, 60e-4, 1e-5), verbose=False)  #  EMBED_DIM=28, UNITS=253: val_N_accuracy: 0.9575 - val_D_accuracy: 0.9856 - val_S_accuracy: 0.9994
    val = history.history["val_N_accuracy"][0]
    print(i, ":", val)
    vals.append(val)
print(np.mean(vals))
model.save_weights('./checkpoints/modern')

0 : 0.9588528871536255
1 : 0.9590473175048828
2 : 0.9547316431999207
3 : 0.9591906070709229
4 : 0.961001992225647
5 : 0.9616599678993225
6 : 0.9608495235443115
7 : 0.9570682048797607
8 : 0.9559904932975769
9 : 0.9598931670188904
10 : 0.9606908559799194
11 : 0.9620185494422913
12 : 0.9541298151016235
13 : 0.9578877687454224
14 : 0.9566827416419983
0.9586463689804077


In [23]:
model.load_weights('./checkpoints/modern')
model.save('modern.h5')

In [9]:
import tensorflowjs as tfjs
# model.load_weights('./checkpoints/modern')
tfjs.converters.save_keras_model(model, '.')

c:\users\elaza\appdata\local\programs\python\python38\lib\site-packages\tensorflowjs\converters\keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [ ]:
model.load_weights('./checkpoints/modern')

def print_predictions(data, s):
    batch = data.normalized[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud[s], data.dagesh[s], data.sin[s]]
    actual = dataset.merge(data.text[s], ts=batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = dataset.merge(data.text[s], ts=batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    total = []
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        last = expected_niqqud[i].tolist().index(0)
        res = expected_niqqud[i][:last] == actual_niqqud[i][:last]
        total.extend(res)
        print(round(np.mean(res), 2), f'({last - sum(res)} out of {last})')
        print()
    print(round(np.mean(total), 3))

print_predictions(data_modern[1], slice(0, None))

In [ ]:
for i in range(50):
    model.load_weights('./checkpoints/pre_modern')
    p1 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    p2 = np.exp(np.random.uniform(low=np.log(1e-4), high=np.log(1e-1)))
    p3 = np.exp(np.random.uniform(low=np.log(1e-5), high=np.log(1e-2)))
    print(p1, p2, p3, end=', ', sep=', ')
    history = fit(data_modern, scheduler=schedulers.CircularLearningRate(p1, p2, p3), verbose=0)
    print(history.history['val_N_accuracy'][0])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()